# Setup

## Import libraries

In [0]:
# Basic libraries
import os
import pandas as pd
import numpy as np
from pprint import pprint
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import json
from google.colab import drive
drive.mount('/content/drive')

# Tokenize
import re
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
# Stopword Removal
import spacy
from spacy.lang.id.stop_words import STOP_WORDS
!python3 -m spacy download en
from nltk.corpus import stopwords
nltk.download('stopwords')

# Vizualization
import PIL
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
%matplotlib inline

## Define funtion

In [0]:
stop_words = list(stopwords.words('english'))
stop_words.extend(['well', 'yeah', 'know', 'okay'])
CONTRACTION_MAP = { "ain't": "is not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'd've": "he would have", "he'll": "he will", "he'll've": "he he will have", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have", "I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have", "so's": "so as", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

expand_contractions("Y'all can't expand contractions I'd think")

## Load data

In [0]:
root = '/The Office'

listdir = [f for f in os.listdir(root+'/data/')]
print(len(listdir))

# Get each character line

In [0]:
character_list = {'Michael':{'mask':'Michael.png', 'lines':[]},
                  'Dwight':{'mask':'Dwight.png', 'lines':[]},
                  'Jim':{'mask':'Jim.png', 'lines':[]},
                  'Pam':{'mask':'Pam.png', 'lines':[]},
                  'Ryan':{'mask':'Ryan.png', 'lines':[]},
                  'Andy':{'mask':'Andy.png', 'lines':[]},
                  'Stanley':{'mask':'Stanley.png', 'lines':[]},
                  'Kevin':{'mask':'Kevin.png', 'lines':[]},
                  'Meredith':{'mask':'Meredith.png', 'lines':[]},
                  'Phyllis':{'mask':'Phyllis.png', 'lines':[]},
                  'Angela':{'mask':'Angela.png', 'lines':[]},
                  'Oscar':{'mask':'Oscar.png', 'lines':[]},
                  'Kelly':{'mask':'Kelly.png', 'lines':[]},
                  'Toby':{'mask':'Toby.png', 'lines':[]},
                  'Creed':{'mask':'Creed.png', 'lines':[]},
                  }

for character in list(character_list.keys()):
    for f in listdir:
        with open(root+'/data/{0}' .format(f)) as data_file:    
            data = json.load(data_file)
            # print('Parsing:', f)
            data_file.close()

        for scene in data['scenes']:
            for line in scene:
                if character in line['character']:
                    l = re.sub('[\(\[].*?[\)\]]', ' ', line['line']).lower() # Remove stage instruction eg. []
                    l = re.sub(' +', ' ', l).strip() # Remove double space and trailing whitespace
                    l = expand_contractions(l) # Expand shortened word
                    l = re.sub('[^a-zA-Z\s]', '', l) # Remove special character and digit
                    l =  [token for token in l.split() if token not in stop_words and len(token)>3]
                    l = ' '.join(l)
                    if l: character_list[character]['lines'].append(l)

# print(json.dumps(data, sort_keys=True, indent=4))

In [0]:
for character in list(character_list.keys()):
    text = ' '.join(character_list[character]['lines'])

    char_mask = np.array(Image.open(root+"/mask/{0}.png" .format(character)))
    image_colors = ImageColorGenerator(char_mask)

    # Create and generate a word cloud image:
    wordcloud = WordCloud(stopwords=stop_words, background_color="white").generate(text)

    # Display the generated image:
    # plt.imshow(wordcloud, interpolation='bilinear')

    wc = WordCloud(background_color="white", width=400, height=400, mask=char_mask).generate(text)
    plt.imshow(wc.recolor(color_func=image_colors))

    print(character)
    plt.axis("off")
    plt.show()